<a href="https://colab.research.google.com/github/em-bst/enhancing-large-language-models-with-human-feedback/blob/main/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install trl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [14]:
""" Full training:
python examples/scripts/reward_modeling.py \
    --model_name_or_path gpt2 \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir ./drive/reward_output \
    --per_device_train_batch_size 4 \
    --num_train_epochs 1 \
    --gradient_checkpointing True \
    --learning_rate 1.0e-5 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 2048 """

' Full training:\npython examples/scripts/reward_modeling.py     --model_name_or_path gpt2     --dataset_name trl-lib/ultrafeedback_binarized     --output_dir ./drive/reward_output     --per_device_train_batch_size 4     --num_train_epochs 1     --gradient_checkpointing True     --learning_rate 1.0e-5     --logging_steps 25     --eval_strategy steps     --eval_steps 50     --max_length 2048 '

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import warnings
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, HfArgumentParser
from datasets import load_dataset, DatasetDict, load_from_disk

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    setup_chat_format,
)

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:

#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"  # adjust the value if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model_name_ = "gpt2"
output_dir = "./drive/reward_output"
dataset_name = "trl-lib/ultrafeedback_binarized"


In [7]:
model_args = ModelConfig(model_name_or_path=model_name_)


In [34]:
#!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [35]:
#!pip install transformers datasets trl accelerate

In [6]:
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

CUDA available: True
CUDA version: 12.1


In [9]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Model
model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path, num_labels=1, trust_remote_code=model_args.trust_remote_code)
model.config.pad_token_id = tokenizer.pad_token_id
#model.to(device)

if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [12]:
dataset = load_dataset("trl-lib/ultrafeedback_binarized")
print(dataset)

README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
})


In [13]:
def tokenize_function(examples):
    """
    Tokenizes and preprocesses the input examples.

    Args:
        examples: A dictionary of examples containing the "chosen" key.

    Returns:
        A dictionary containing the tokenized input IDs, attention mask, etc.
    """
    return tokenizer(
        [str(x) for x in examples["chosen"]],
        truncation=True,  # Truncate sequences longer than max_length
        padding="max_length",  # Pad shorter sequences to max_length
        max_length=min(tokenizer.model_max_length, 200),  # Set max sequence length (reduced for CUDA memory)
        return_tensors="pt"  # Return PyTorch tensors
    )

# Apply the tokenize_function to the dataset using the `map` function
tokenized_dataset = dataset.map(
    tokenize_function,  # The function to apply
    batched=True,  # Process examples in batches for efficiency
  # Use 4 processes for parallel processing
)

Map:   0%|          | 0/1242 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids', 'attention_mask'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [13]:
def tokenize_function(batch):
    """DESCRIBE DOCSTRING"""
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(batch["chosen"], batch["rejected"]):
        tokenized_chosen = tokenizer(str(chosen), padding="max_length", truncation=True, return_tensors="pt")
        tokenized_rejected = tokenizer(str(rejected), padding="max_length", truncation=True, return_tensors="pt")
        new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

    return new_examples

# Apply to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 1000
    })
})

In [12]:
tokenized_datasets.save_to_disk('/content/drive/MyDrive/tokenized_datasets')

Saving the dataset (0/5 shards):   0%|          | 0/62135 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#tokenized_datasets = load_from_disk('/content/drive/MyDrive/tokenized_datasets')

In [11]:
# def tokenize_function(batch):
#     """DESCRIBE DOCSTRING"""
#     new_examples = {
#         "input_ids_chosen": [],
#         "attention_mask_chosen": [],
#         "input_ids_rejected": [],
#         "attention_mask_rejected": [],
#     }
#     for chosen, rejected in zip(batch["chosen"], batch["rejected"]):
#         # Tokenize chosen and rejected separately
#         tokenized_chosen = tokenizer(str(chosen), padding="max_length", truncation=True, return_tensors="pt")
#         tokenized_rejected = tokenizer(str(rejected), padding="max_length", truncation=True, return_tensors="pt")

#         # Ensure attention masks have correct values (0 for padding, 1 for actual tokens)
#         tokenized_chosen["attention_mask"] = tokenized_chosen["attention_mask"].type(torch.int64) # Enforce int64 type for attention mask
#         tokenized_rejected["attention_mask"] = tokenized_rejected["attention_mask"].type(torch.int64) # Enforce int64 type for attention mask

#         # Append to new_examples
#         new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
#         new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
#         new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
#         new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

#     return new_examples

# # Apply the modified tokenize_function to the dataset
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
train_subset_indices = torch.randint(0, len(tokenized_datasets["train"]), size=(int(0.02 * len(tokenized_datasets["train"])),))
train_subset = tokenized_datasets["train"].select(train_subset_indices.tolist())

# Update the dataset with the new subset
tokenized_datasets['train'] = train_subset

In [16]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 1242
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 1000
    })
})

In [24]:
train_subset = tokenized_datasets['train'].shuffle(seed=42).select(range(int(0.02 * len(tokenized_datasets['train']))))

# Update the dataset with the new subset
tokenized_datasets['train'] = train_subset

# Check the new size
print(f"Number of training rows after sampling: {len(tokenized_datasets['train'])}")

Number of training rows after sampling: 1242


In [ ]:
# #Tokenisation

# def tokenize_function(data):
#   """ NE PAS OUBLIER LE DOCSTRING"""
#   chosen_tokens = tokenizer([str(chosen) for chosen in data["chosen"]], padding="max_length", truncation=True, return_tensors="pt")
#   rejected_tokens = tokenizer([str(rejected) for rejected in data["rejected"]], padding="max_length", truncation=True, return_tensors="pt")
#   return {
#       "chosen": data["chosen"],
#       "rejected": data["rejected"],
#       "score_chosen": data["score_chosen"],
#       "score_rejected": data["score_rejected"],
#       "input_ids_chosen": chosen_tokens.input_ids,
#       "attention_mask_chosen": chosen_tokens.attention_mask,
#       "input_ids_rejected": rejected_tokens.input_ids,
#       "attention_mask_rejected": rejected_tokens.attention_mask,
#   }

# tokenized_datasets = dataset.map(tokenize_function, batched = True) # Use the dataset for tokenization


Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 1242
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 1000
    })
})

In [1]:
script_args = ScriptArguments(
    dataset_name=dataset_name,
    dataset_train_split="train",
    dataset_test_split="test")

training_args = RewardConfig(
    output_dir="/content/drive/MyDrive/reward_output",           # Répertoire de sortie
    per_device_train_batch_size=8,               # Taille de batch par périphérique
    num_train_epochs=1,                          # Nombre d'époques
    gradient_checkpointing=False,                 # Activer Gradient Checkpointing
    learning_rate=1.0e-5,                        # Taux d'apprentissage
    logging_steps=25,                            # Intervalle pour afficher les logs
    evaluation_strategy="steps",                 # Évaluer à chaque `eval_steps`
    eval_steps=50,                               # Intervalle pour évaluation                             # Pas de limite explicite sur les étapes
    report_to = "none"
)

NameError: name 'ScriptArguments' is not defined

In [ ]:

##########
# Training
##########
trainer = RewardTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets[script_args.dataset_train_split],
    eval_dataset=tokenized_datasets[script_args.dataset_test_split],
    )
trainer.train()

# trainer = RewardTrainer(
#     args=reward_config,  # Training arguments
#     model=model,  # The model to train
#     tokenizer=tokenizer,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset['test'],
#     compute_metrics=compute_metrics,
# )

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
assert sequence_lengths.max() < logits.size(1), "sequence_lengths contains invalid indices"


NameError: name 'sequence_lengths' is not defined

In [ ]:
############################
    # Save model and push to Hub
    ############################
    trainer.save_model(training_args.output_dir)

    if training_args.eval_strategy != "no":
        metrics = trainer.evaluate()
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    # Save and push to hub
    trainer.save_model(training_args.output_dir)
    if training_args.push_to_hub:
        trainer.push_to_hub(dataset_name=script_args.dataset_name)

In [ ]:
# 0. imports
import torch
from transformers import GPT2Tokenizer

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer


# 1. load a pretrained model
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# 2. initialize trainer
ppo_config = {"mini_batch_size": 1, "batch_size": 1}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer)

# 3. encode a query
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

# 4. generate model response
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 20,
}
response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_kwargs)
response_txt = tokenizer.decode(response_tensor[0])

# 5. define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0, device=model.pretrained_model.device)]

# 6. train model with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [ ]:
# ... (your existing code for loading model, tokenizer, datasets) ...

# 1. Create a reference model (copy of the original model)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code
).to(device)
ref_model.config.pad_token_id = tokenizer.pad_token_id
if tokenizer.chat_template is None:
    ref_model, tokenizer = setup_chat_format(ref_model, tokenizer)

# 2. Initialize PPO trainer
ppo_config = PPOConfig(mini_batch_size=4, batch_size=4)  # Adjust batch sizes
ppo_trainer = PPOTrainer(ppo_config, model, ref_model, tokenizer)

# 3. Training loop (integrate within your existing training logic)
for epoch in range(training_args.num_train_epochs):
    for batch in tokenized_datasets["train"]:  # Iterate through your training dataset
        # a. Prepare query and response tensors
        query_tensor = batch["input_ids_chosen"].to(device)
        response_tensor = batch["input_ids_rejected"].to(device)

        # b. Define reward (replace with your reward function)
        reward = [
            torch.tensor(1.0, device=device)
        ]  # Replace with your actual reward calculation

        # c. Train with PPO
        train_stats = ppo_trainer.step(query_tensor, response_tensor, reward)

# ... (rest of your code for saving, evaluation, etc.) ...